## Microservices Model

© 2020 By The Rector And Visitors Of The University Of Virginia

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
import requests 
import json
import FAIR

## First get authorized

Visit:
<br>
https://clarklab.uvarc.io/auth/login
<br>
And copy and paste access token below

In [2]:
token =

## Using Transfer Service upload data with metadata

Upload a file to minio via the upload_file function.
<br>
<br>
upload_file takes:
        - file_path: path to local file 
        - metadata: describes the file to be uploaded
returns:
    - PID representing object
upload_file then calls the transfer service which uploads the file to minio and metadata to mongoDB and Stardog

In [3]:
dataset_meta = {
    "@context":{
        "@vocab":"http://schema.org/"
    },
    "@type":"Dataset",
    "name":"Raw Time Series Feature Set",
    "description":"Heart Rate Measures from patient from admission to discharge.",
    "author":[
        {
            "name":"Justin Niestroy",
            "@id": "https://orcid.org/0000-0002-1103-3882",
            "@type":"Person",
            "affiliation":"University of Virginia"
        }
    ],
}
data_id = FAIR.upload_file('UVA_7129_HR2.csv',dataset_meta,token = token)
data_id

'ark:99999/bbe4cbef-c073-49c9-baa6-90c52fc2b63d'

## Check MDS to make sure metadata was uploaded correctly

retrieve_metadata calls MDS to get the metadata for the newly minted id.
<br>
<br>
retrieve_metadata takes:
    - PID: identifier of interest
returns:
    - json-ld metadata of given PID

In [4]:
FAIR.retrieve_metadata(data_id,token = token)

{'@context': {'@vocab': 'http://schema.org/'},
 '@id': 'ark:99999/bbe4cbef-c073-49c9-baa6-90c52fc2b63d',
 '@type': 'Dataset',
 'author': [{'@id': 'https://orcid.org/0000-0002-1103-3882',
   'affiliation': 'University of Virginia',
   'name': 'Justin Niestroy'}],
 'description': 'Heart Rate Measures from patient from admission to discharge.',
 'distribution': [{'@id': 'ark:99999/e2a2996a-d0f8-41fa-8972-1c9fe331eb50'}],
 'name': 'Raw Time Series Feature Set',
 'sdPublicationDate': '2021-02-02T16:40:29.649338829Z',
 'sdPublisher': {'@id': 'e384977d-2b5a-493a-b3ac-a9e981806ba8',
  'name': 'Justin Admin'},
 'url': 'http://ors.uvadcos.io/ark:99999/bbe4cbef-c073-49c9-baa6-90c52fc2b63d'}

## Upload Spark Script to run on newly uploaded data

In [5]:
spark_code_meta = {
    "@context":{
        "@vocab":"http://schema.org/"
    },
    "@type":"SoftwareSourceCode",
    "name":"Raw Data Analysis Script",
    "description":"Sample Source Code for HCTSA test on spark",
    "author":[
        {
            "name":"Justin Niestroy",
            "@id": "https://orcid.org/0000-0002-1103-3882",
            "@type":"Person",
            "affiliation":"University of Virginia"
      }
    ]
}
software_id = FAIR.upload_file('run_algos.py',spark_code_meta,token = token)
software_id

'ark:99999/2bb5174f-4558-4c69-b339-25c1ebd13c10'

## Submit Spark Job Using Compute Service

Run a the uploaded script on the uploaded data by calling compute
<br>
<br>
FAIR.compute takes: 
    - data pid 
    - software pid
    - type of job ie spark,nipype,custom  
compute then launches kubernetes pods to run the given script on the provided data using specialized pods
<br>
<br>
compute returns:
    - job_PID 

In [6]:
job_id = FAIR.compute(data_id,software_id,'spark',token = token)
job_id

'ark:99999/d24ba632-71cf-4552-a103-9f729850daf8'

## Watch Running Job
list_running_jobs takes:
<br>
<br>
list_running_jobs returns:
    - list of running job PIDs

In [9]:
FAIR.list_running_jobs(token = token)

[]

## Get Job outputs from job metadata

Upon completion of a job the compute service updates the job id metadata to include properties:
    - evi:supports: list of output PIDs 
    - logs: logs from job

In [10]:
job_metadata = FAIR.retrieve_metadata(job_id,token = token)
output_ids = job_metadata.get('evi:supports')
output_ids

[{'@id': 'ark:99999/17f3abc9-6883-40f1-9e2e-1381cda5ad2b'},
 {'@id': 'ark:99999/f94fa455-79dd-4a19-aab8-4e4bef40f12e'}]

The computation created 2 outputs
<br>
Metadata for the two created outputs is shown below

In [11]:
output_metadata = [ 
   FAIR.retrieve_metadata(out['@id'],token = token) for out in output_ids
]
output_metadata

[{'@context': {'@vocab': 'http://schema.org/'},
  '@id': 'ark:99999/17f3abc9-6883-40f1-9e2e-1381cda5ad2b',
  '@type': 'Dataset',
  'distribution': [{'@id': 'ark:99999/98a46d5d-c302-452a-a019-990bfb118698',
    '@type': 'Download',
    'contentUrl': 'minio:9000/breakfast/d24ba632-71cf-4552-a103-9f729850daf8/_SUCCESS',
    'fileFormat': '_SUCCESS',
    'name': 'breakfast/d24ba632-71cf-4552-a103-9f729850daf8/_SUCCESS'}],
  'evi:generatedBy': {'@id': 'ark:99999/d24ba632-71cf-4552-a103-9f729850daf8'},
  'name': '_SUCCESS',
  'sdPublicationDate': '2021-02-02T16:41:11.990963925Z',
  'sdPublisher': {'@id': 'e384977d-2b5a-493a-b3ac-a9e981806ba8',
   'name': 'Justin Admin'},
  'url': 'http://ors.uvadcos.io/ark:99999/17f3abc9-6883-40f1-9e2e-1381cda5ad2b'},
 {'@context': {'@vocab': 'http://schema.org/'},
  '@id': 'ark:99999/f94fa455-79dd-4a19-aab8-4e4bef40f12e',
  '@type': 'Dataset',
  'distribution': [{'@id': 'ark:99999/038a475e-ee96-4e3f-b725-3adf2b7bdcfd',
    '@type': 'Download',
    'contentU

## Upload Image Code

In [12]:
spark_graphic_meta = {
    "@context":{
        "@vocab":"http://schema.org/"
    },
    "@type":"SoftwareSourceCode",
    "name":"HCTSA Cluster Script",
    "description":"Creates heatmap of patients stay",
    "author":[
        {
            "name":"Justin Niestroy",
            "@id": "https://orcid.org/0000-0002-1103-3882",
            "affiliation":"University of Virginia"
      }
    ]
}
graphic_code_id = FAIR.upload_file('make_graphic.py',spark_graphic_meta,token = token)

## Run Spark Job to create Image

In [13]:
image_job_id = FAIR.compute(output_ids[1]['@id'],graphic_code_id,'spark',token = token)
image_job_id

'ark:99999/bea54627-e761-4c29-b9c0-4ba60f21e3e0'

## Check on running jobs

In [15]:
FAIR.list_running_jobs(token = token)

[]

### Get image ID from Updated Job ID

In [16]:
image_job_meta = FAIR.retrieve_metadata(image_job_id,token = token)
image_id = image_job_meta['evi:supports'][0]['@id']

## Get the image metadata

In [17]:
FAIR.retrieve_metadata(image_id,token = token)

{'@context': {'@vocab': 'http://schema.org/'},
 '@id': 'ark:99999/ee797b68-57ac-407d-b995-dbd820283f08',
 '@type': 'Dataset',
 'distribution': [{'@id': 'ark:99999/2e841cf0-0b7d-4b53-8e22-8bdcc7027aad',
   '@type': 'Download',
   'contentUrl': 'minio:9000/breakfast/bea54627-e761-4c29-b9c0-4ba60f21e3e0/HCTSA Heatmap.png',
   'fileFormat': 'png',
   'name': 'breakfast/bea54627-e761-4c29-b9c0-4ba60f21e3e0/HCTSA Heatmap.png'}],
 'evi:generatedBy': {'@id': 'ark:99999/bea54627-e761-4c29-b9c0-4ba60f21e3e0'},
 'name': 'HCTSA Heatmap.png',
 'sdPublicationDate': '2021-02-02T16:47:44.21205201Z',
 'sdPublisher': {'@id': 'e384977d-2b5a-493a-b3ac-a9e981806ba8',
  'name': 'Justin Admin'},
 'url': 'http://ors.uvadcos.io/ark:99999/ee797b68-57ac-407d-b995-dbd820283f08'}

### Build Evidence graph of created image using the evidence graph service

Evidence Graph Service builds an json-ld evidence graph representing the provenance (datasets, software, and computations that support the PID)
<br>
<br>
evidence_graph takes:
    - PID
evidence_graph returns:
    - json-ld evidence graph for given PID

In [18]:
FAIR.evidence_graph(image_id,token = token)

{'@context': {'@vocab': 'http://schema.org/', 'evi': 'http://w3id.org/EVI#'},
 '@id': 'ark:99999/ee797b68-57ac-407d-b995-dbd820283f08',
 '@type': 'Dataset',
 'evi:generatedBy': {'@id': 'ark:99999/bea54627-e761-4c29-b9c0-4ba60f21e3e0',
  '@type': 'evi:Computation',
  'evi:usedDataset': {'@id': 'ark:99999/f94fa455-79dd-4a19-aab8-4e4bef40f12e',
   '@type': 'Dataset',
   'evi:generatedBy': {'@id': 'ark:99999/d24ba632-71cf-4552-a103-9f729850daf8',
    '@type': 'evi:Computation',
    'evi:usedDataset': {'@id': 'ark:99999/bbe4cbef-c073-49c9-baa6-90c52fc2b63d',
     '@type': 'Dataset',
     'author': {'@id': 'https://orcid.org/0000-0002-1103-3882',
      'name': 'Justin Niestroy'},
     'name': 'Raw Time Series Feature Set'},
    'evi:usedSoftware': {'@id': 'ark:99999/2bb5174f-4558-4c69-b339-25c1ebd13c10',
     '@type': 'SoftwareSourceCode',
     'author': {'@id': 'https://orcid.org/0000-0002-1103-3882',
      'name': 'Justin Niestroy'},
     'name': 'Raw Data Analysis Script'},
    'name': 'C

## View Visualization of the Evidence Graph

In [20]:
'https://clarklab.uvarc.io/viz/' + image_id

'https://clarklab.uvarc.io/viz/ark:99999/ee797b68-57ac-407d-b995-dbd820283f08'

## Download Created Image

Download output files from computations using download_file
<br>
<br>
download_file takes:
    - PID: id representing object to be downloaded
    - name: file name for downloaded object
<br>
download_file returns:<br>
         - bool: True if download if succesful<br>
         - file saved location given

In [19]:
from IPython.display import Image
FAIR.download_file(image_id,'Created Image.png',token = token)
Image(filename='Created Image.png') 

In [ ]:
r = requests.post('https://clarklab.uvarc.io/auth/logout',headers = {"Authorization":token })
r.content.decode()